In [ ]:
import yaml
import networkx as nx
import pygraphviz as pgv
import pandas as pd
from pulp import LpProblem, LpMaximize, lpSum
from ortoolpy import addbinvar, addvals

In [ ]:
def create_gradient():
    colors = [
        '#e53935',
        '#e6353b',
        '#e63241',
        '#e62e47',
        '#e62b4d',
        '#e52852',
        '#e52658',
        '#e4255d',
        '#e22463',
        '#e12368',
        '#df246e',
        '#dc2573',
        '#da2678',
        '#d7287d',
        '#d42a82',
        '#d02d87',
        '#cd308c',
        '#c93391',
        '#c43695',
        '#c03999',
        '#bb3c9e',
        '#b63fa1',
        '#b142a5',
        '#ab45a9',
        '#a548ac',
        '#9f4aaf',
        '#994db2',
        '#9250b5',
        '#8b52b7',
        '#8454b9',
        '#7d56bb',
        '#7559bd',
        '#6c5abe',
        '#645cbf',
        '#5b5ec0',
        '#5160c0',
        '#4661c1',
        '#3a63c1',
        '#2b64c0',
        '#1565c0',
    ]
    colors2 = [
        '#e53935',
        '#e23e30',
        '#e0422c',
        '#dd4627',
        '#da4a22',
        '#d74d1d',
        '#d45118',
        '#d15413',
        '#ce580c',
        '#ca5b05',
        '#c75e00',
        '#c36000',
        '#c06300',
        '#bc6600',
        '#b86800',
        '#b46b00',
        '#b16d00',
        '#ad6f00',
        '#a97100',
        '#a57300',
        '#a17500',
        '#9d7700',
        '#997800',
        '#957a00',
        '#917c00',
        '#8d7d00',
        '#897e00',
        '#868000',
        '#828100',
        '#7e8200',
        '#7a8300',
        '#768404',
        '#72850c',
        '#6e8612',
        '#6a8718',
        '#66881d',
        '#618922',
        '#5d8a26',
        '#598a2b',
        '#558b2f',
    ]
    return ':'.join([color + ';0.025' for color in colors])

In [ ]:
def load_catalog():
    with open('./aws-catalog.yml') as file:
        return yaml.load(file, Loader=yaml.FullLoader)
    
def load_service_names():
    catalog = load_catalog()
    services = catalog['services']
    service_names = [service['acronym'] if service['acronym'] is not None else service['name'] for service in services]
    return service_names

def load_acronyms():
    catalog = load_catalog()
    services = catalog['services']
    services_with_acronym = list(filter(lambda service: service['acronym'] is not None, services))
    terms_with_acronym = catalog['acronyms']
    words_with_acronym = services_with_acronym + terms_with_acronym
    acronyms = [word['acronym'] for word in words_with_acronym]
    return acronyms

In [ ]:
def get_nodes_from_word(word):
    return (word[0].upper(), word[-1].upper())

def build_graph_from_all_edges(words):
    graph = nx.MultiDiGraph()
    graph.add_nodes_from(['start', 'end'])
    for word in words:
        (start, end) = get_nodes_from_word(word)
        graph.add_edge(start, end, word=word, var=addbinvar())
    
    # Create edges from 'start'
    # AWSしりとり starts from 'S' because 'AWS' precedes the first word
    graph.add_edge('start', 'S', word='', var=addbinvar())
    
    # Create edges to 'end'
    for node in list(graph.nodes)[2:]:
        graph.add_edge(node, 'end', word='', var=addbinvar())
    return graph

def build_dataframe_from_graph(graph):
    df = pd.DataFrame([(from_node, to_node, key, data['word'], data['var'])
        for (from_node, to_node, key), data in graph.edges.items()],
        columns=['From', 'To', 'Key', 'Word', 'Var'])
    return df

def build_problem(graph, df):
    problem = LpProblem(sense=LpMaximize)
    
    # Objective: Maximize the number of edges
    problem += lpSum(df.Var)
    
    # Constraint: Only one edge from 'start' and to 'end' respectively
    problem += lpSum(df[df.From == 'start'].Var) == 1
    problem += lpSum(df[df.To == 'end'].Var) == 1
    
    # Constraint: Any character node (not 'start' and 'end' node)
    #   has the same number of edges that go into it and out of it
    char_nodes = list(graph.nodes())[2:] # Nodes excluding 'start' and 'end'
    for node in char_nodes:
        problem += (lpSum([to_edge[2] for to_edge in graph.in_edges(node, data='var')])
           == lpSum([from_edge[2] for from_edge in graph.edges(node, data='var')]))
    return problem

def build_graph_from_used_edges(df):
    graph = nx.MultiDiGraph()
    addvals(df)
    for row in df[df.Val > 0.5].itertuples():
        graph.add_edge(row.From, row.To, word=row.Word)
    return graph

def get_route(graph):
    graph.add_edge('end', 'start') # Make the path an eulerian circuit
    circuit = nx.eulerian_circuit(graph, 'start', True)
    edges = list(circuit)[1:-2] # Exclude 'start' and 'end'
    route = [graph[from_node][to_node][key]['word'] for from_node, to_node, key in edges]
    return route

def visualize_graph(original_graph):
    graph = original_graph.copy()
    
    # Remove unnecessary nodes
    graph.remove_node('start')
    graph.remove_node('end')
    
    # Add styles to nodes
    for node in graph.nodes():
        graph.nodes[node]['shape'] = 'circle'
        graph.nodes[node]['fontsize'] = '48'
        graph.nodes[node]['fontname'] = 'arial'
    
    # Add styles to edges
    for from_node, to_node, key, data in graph.edges(keys=True, data=True):
        # graph.edges[from_node, to_node, key]['label'] = data['word'] # Too messy
        graph.edges[from_node, to_node, key]['color'] = create_gradient()
        graph.edges[from_node, to_node, key]['penwidth'] = '3'
    
    agraph = nx.nx_agraph.to_agraph(graph)
    agraph.draw('outputs/file.svg', format='svg', prog='circo')

In [ ]:
words = load_service_names()
all_edges_graph = build_graph_from_all_edges(words)
df = build_dataframe_from_graph(all_edges_graph)
problem = build_problem(all_edges_graph, df)
problem.solve()
used_edges_graph = build_graph_from_used_edges(df)
route = get_route(used_edges_graph)
print(len(route), route)
visualize_graph(all_edges_graph)